<a href="https://colab.research.google.com/github/ashtheflash1212/TensorFlow-Pytorch/blob/main/Pytorch_using_2022_2023_NBA_Player_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np


In [2]:
import zipfile

zf = zipfile.ZipFile('/content/drive/MyDrive/Colab Notebooks/archive (11).zip')
df = pd.read_csv(zf.open('nba_2022-23_all_stats_with_salary.csv'))

df.head()



,Unnamed: 0,Player Name,Salary,Position,Age,Team,GP,GS,MP,FG,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,0,Stephen Curry,48070014,PG,34,GSW,56,56,34.7,10.0,...,12.5,31.0,5.8,2.0,7.8,0.192,7.5,0.1,7.5,4.7
1,1,John Wall,47345760,PG,32,LAC,34,3,22.2,4.1,...,17.1,27.0,-0.4,0.7,0.3,0.020,-0.8,-0.4,-1.2,0.1
2,2,Russell Westbrook,47080179,PG,34,LAL/LAC,73,24,29.1,5.9,...,18.4,27.7,-0.6,2.6,1.9,0.044,0.3,-0.1,0.2,1.2
3,3,LeBron James,44474988,PF,38,LAL,55,54,35.5,11.1,...,11.6,33.3,3.2,2.4,5.6,0.138,5.5,0.6,6.1,4.0
4,4,Kevin Durant,44119845,PF,34,BRK/PHO,47,47,35.6,10.3,...,13.4,30.7,4.7,2.1,6.8,0.194,6.0,1.2,7.1,3.9


In [ ]:
print(df.index)

RangeIndex(start=0, stop=467, step=1)


In [ ]:
print(df.columns.tolist())

['Unnamed: 0', 'Player Name', 'Salary', 'Position', 'Age', 'Team', 'GP', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Total Minutes', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 467 entries, 0 to 466
Data columns (total 52 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     467 non-null    int64  
 1   Player Name    467 non-null    object 
 2   Salary         467 non-null    int64  
 3   Position       467 non-null    object 
 4   Age            467 non-null    int64  
 5   Team           467 non-null    object 
 6   GP             467 non-null    int64  
 7   GS             467 non-null    int64  
 8   MP             467 non-null    float64
 9   FG             467 non-null    float64
 10  FGA            467 non-null    float64
 11  FG%            466 non-null    float64
 12  3P             467 non-null    float64
 13  3PA            467 non-null    float64
 14  3P%            454 non-null    float64
 15  2P             467 non-null    float64
 16  2PA            467 non-null    float64
 17  2P%            463 non-null    float64
 18  eFG%      

In [3]:
from sklearn.model_selection import train_test_split

attributes = df.drop(['Player Name', 'Position', 'Team', 'Salary'],axis=1)
target = df['Salary']

attributes.astype({col: float for col in attributes.columns[1:]})

X = torch.tensor(attributes.values, dtype=torch.float)
y = torch.tensor(target.values, dtype=torch.float).view(-1,1)

X_np = X.numpy()
y_np = y.numpy()  # Make sure y is also in NumPy format

# Create a mask for rows without NaNs
mask = ~np.isnan(X_np).any(axis=1)

# Filter X and y
X_clean = X_np[mask]
y_clean = y_np[mask]

# Convert back to tensors
X = torch.tensor(X_clean, dtype=torch.float32)
y = torch.tensor(y_clean, dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=2)



if torch.isnan(X_train).any() or torch.isinf(X_train).any():
    print("X_train contains NaNs or Infs")
if torch.isnan(y_train).any() or torch.isinf(y_train).any():
    print("y_train contains NaNs or Infs")



In [4]:
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

In [5]:
torch.manual_seed(42)
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.01)
loss = nn.MSELoss()


num_epochs = 5000
for epoch in range(num_epochs):
    predictions = model(X_train)
    MSE = loss(predictions, y_train)
    MSE.backward()
    optimizer.step()
    optimizer.zero_grad()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], MSE Loss: {MSE.item()}')

Epoch [100/5000], MSE Loss: 78629962776576.0
Epoch [200/5000], MSE Loss: 66796841336832.0
Epoch [300/5000], MSE Loss: 49109461368832.0
Epoch [400/5000], MSE Loss: 19280389210112.0
Epoch [500/5000], MSE Loss: 13240728813568.0
Epoch [600/5000], MSE Loss: 9492174471168.0
Epoch [700/5000], MSE Loss: 6247553171456.0
Epoch [800/5000], MSE Loss: 3767932813312.0
Epoch [900/5000], MSE Loss: 2346575200256.0
Epoch [1000/5000], MSE Loss: 1658526105600.0
Epoch [1100/5000], MSE Loss: 1316738564096.0
Epoch [1200/5000], MSE Loss: 1119198248960.0
Epoch [1300/5000], MSE Loss: 985881772032.0
Epoch [1400/5000], MSE Loss: 890401521664.0
Epoch [1500/5000], MSE Loss: 814899855360.0
Epoch [1600/5000], MSE Loss: 752522821632.0
Epoch [1700/5000], MSE Loss: 698276249600.0
Epoch [1800/5000], MSE Loss: 651998265344.0
Epoch [1900/5000], MSE Loss: 609480802304.0
Epoch [2000/5000], MSE Loss: 571300904960.0
Epoch [2100/5000], MSE Loss: 537542131712.0
Epoch [2200/5000], MSE Loss: 504981487616.0
Epoch [2300/5000], MSE L

In [9]:
y_pred = model(X_test)

print(y_pred[:5])
print(y_test[:5])

tensor([[34376036.0000],
        [ 3563034.7500],
        [ 2557738.7500],
        [ 2149327.0000],
        [ 5539517.0000]], grad_fn=<SliceBackward0>)
tensor([[35448672.],
        [ 4215120.],
        [ 2696400.],
        [ 2298385.],
        [ 5722116.]])


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    attributes, target, test_size=0.33, random_state=2)
forest = RandomForestClassifier(n_estimators=20, random_state=1)
forest.fit(X_train, y_train)
print(forest.score(X_test, y_test))


0.07096774193548387


In [ ]:
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/london_weather.csv')
df2.head()
#temp in Celsius, #sunshine measurement in hours

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,19790104,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,19790105,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

df2 = df2.dropna()

target = df2['mean_temp']
attributes = df2.drop(['max_temp','min_temp', 'mean_temp','date', 'snow_depth'],axis=1)


from sklearn.ensemble import RandomForestRegressor # Changed from RandomForestClassifier to RandomForestRegressor
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    attributes, target, test_size=0.33, random_state=1)

forest = RandomForestRegressor(n_estimators=20, random_state=1)

forest.fit(X_train, y_train)

y_pred = forest.predict(X_test)

mse_best = mean_squared_error(y_test, y_pred)
mae_best = mean_absolute_error(y_test, y_pred)
r2_best = r2_score(y_test, y_pred)

print(f"Optimized Mean Squared Error: {mse_best}")
print(f"Optimized Mean Absolute Error: {mae_best}")
print(f"Optimized R-squared: {r2_best}")

Optimized Mean Squared Error: 15.664672269197917
Optimized Mean Absolute Error: 3.16997218574451
Optimized R-squared: 0.5203519787157584
